# EDA of Metro HubNashville Dataset

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
##import seaborn as sns
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster
from shapely.geometry import Point

%matplotlib inline

## Read in HubNashville data file

In [2]:
hub_data = pd.read_csv('../data/hubNashville__311__Service_Requests.csv', low_memory = False)

## My Questions
- what to do with nulls?
- fix date format?
- separate time from date column?


- longest open request?
- how quickly are most resolved?
- most common request type?
- what time do most requests come in?
- what month do most requests come in? 
- what council district has the most requests?    
- how many requests are made in a language other than English? No values in this column?  
- how many resolved the same day, in a week?
- how many escalated?
- What are the COVID-19 requests about?
- How has COVID-19 impacted (or will impact) public transportation? Animal control?

## Questions our team would like to answer:
- Trending requests: daily, weekly, monthly, yearly, time of day. Show most demanding requests trends daily, weekly, monthly... and show a comparison  over  the last 3 years, perhaps divide 24 hours into subsets. To possibly prepare for up coming request types and get an overview before digging deeper.
- Covid-19 impact (requests, violations) by council district
- Tornado impact - Hands On volunteers -- anything to improve for the next need?
- Parent requests - how many ‘child’ requests are added while the issue is unresolved.
- Average resolution time by type of request (this could be provided to people who make requests, so they have an ETA )


## Preliminary look at the data

In [3]:
hub_data.head()

,Request #,Status,Request Type,Subrequest Type,Additional Subrequest Type,Date / Time Opened,Date / Time Closed,Request Origin,System of Origin ID,Contact Type,...,State Issue,Parent Request,Closed When Created,Address,City,Council District,ZIP,Latitude,Longitude,Mapped Location
0,353977,Closed,COVID-19,COVID-19 - Submit Input about The Plan to Reop...,COVID-19 - Submit Input about The Plan to Reop...,04/28/2020 02:33:22 PM,04/28/2020 02:33:22 PM,hubNashville Community,NaN,NaN,...,False,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,354524,Closed,Quick Requests,Quick Requests,NaN,04/28/2020 09:46:53 PM,04/28/2020 09:46:53 PM,Phone,NaN,NaN,...,False,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,354188,Closed,Quick Requests,Quick Requests,NaN,04/28/2020 05:09:02 PM,04/28/2020 05:09:02 PM,Phone,NaN,NaN,...,False,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,354445,Closed,Quick Requests,Quick Requests,NaN,04/28/2020 08:50:36 PM,04/28/2020 08:50:36 PM,Phone,NaN,NaN,...,False,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,334893,Closed,Public Safety,Control Number Request for Towing,NaN,03/29/2020 01:08:55 AM,03/29/2020 01:08:55 AM,hubNashville Community,NaN,NaN,...,False,NaN,True,Long Hollow Pike & Northcreek Blvd,GOODLETTSVILLE,10.0,37072,36.32619,-86.700618,"(36.32619, -86.70061799999999)"


In [4]:
hub_data.shape

(348671, 21)

In [5]:
hub_data.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 348671 entries, 0 to 348670
Data columns (total 21 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   Request #                   348671 non-null  int64  
 1   Status                      348671 non-null  object 
 2   Request Type                348603 non-null  object 
 3   Subrequest Type             338024 non-null  object 
 4   Additional Subrequest Type  180510 non-null  object 
 5   Date / Time Opened          348671 non-null  object 
 6   Date / Time Closed          338424 non-null  object 
 7   Request Origin              348671 non-null  object 
 8   System of Origin ID         26398 non-null   object 
 9   Contact Type                26129 non-null   object 
 10  Preferred Language          0 non-null       float64
 11  State Issue                 348671 non-null  bool   
 12  Parent Request              3589 non-null    float64
 13  Closed When Cr

## Sort values by when request was opened

In [6]:
hub_data.sort_values(by = 'Date / Time Opened')

,Request #,Status,Request Type,Subrequest Type,Additional Subrequest Type,Date / Time Opened,Date / Time Closed,Request Origin,System of Origin ID,Contact Type,...,State Issue,Parent Request,Closed When Created,Address,City,Council District,ZIP,Latitude,Longitude,Mapped Location
317237,35966,Closed,"Trash, Recycling & Litter",Cart Service,Trash - Report a Damaged Cart,01/01/2018 02:53:00 PM,01/03/2018 03:02:00 PM,hubNashville Community,NaN,Resident,...,False,NaN,True,1015 Maplewood Pl,NASHVILLE,8.0,37216,36.227550,-86.727430,"(36.22755, -86.72743)"
315401,35967,Closed,"Trash, Recycling & Litter",Recycling Drop Off Sites,NaN,01/01/2018 03:08:00 PM,01/02/2018 08:51:00 AM,hubNashville Community,NaN,Resident,...,False,NaN,True,1333 Lovejoy Court,NASHVILLE,7.0,37216,36.206520,-86.729020,"(36.20652, -86.72902)"
327939,35968,Closed,"Trash, Recycling & Litter",Cart Service,Trash - Request an Additional Cart,01/01/2018 03:14:00 PM,04/17/2018 04:00:00 AM,hubNashville Community,NaN,Resident,...,False,NaN,True,907 N 16th St,NASHVILLE,6.0,37206,36.186170,-86.740370,"(36.18617, -86.74037)"
323146,35969,Closed,"Trash, Recycling & Litter",Cart Service,Recycling - Request an Additional Cart,01/01/2018 03:55:00 PM,01/10/2018 04:04:00 AM,hubNashville Community,NaN,Resident,...,False,NaN,True,2930 Lakeland Dr,NASHVILLE,14.0,37214,36.153670,-86.655230,"(36.15367, -86.65523)"
318265,35970,Closed,"Trash, Recycling & Litter",Cart Service,Recycling - Request an Additional Cart,01/01/2018 04:44:00 PM,01/04/2018 04:02:00 AM,hubNashville Community,NaN,Resident,...,False,NaN,True,106 Creighton Ave,NASHVILLE,6.0,37206,36.190940,-86.729430,"(36.19094, -86.72943)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75878,286840,Closed,Property Violations,Junk and Debris,NaN,12/31/2019 12:37:00 AM,01/24/2020 08:09:00 AM,Mobile App,NaN,NaN,...,False,NaN,False,960 44th Ave,NASHVILLE,20.0,37209,36.160719,-86.840244,"(36.16071861, -86.84024379)"
77255,286841,Closed,Property Violations,Junk and Debris,NaN,12/31/2019 12:40:00 AM,01/31/2020 08:08:00 AM,Mobile App,NaN,NaN,...,False,NaN,False,3824 Dr Walter S Davis Blvd,NASHVILLE,21.0,37209,36.166367,-86.835284,"(36.16636662, -86.83528438)"
80594,286842,Closed,"Streets, Roads & Sidewalks",Roadwork Complaint,NaN,12/31/2019 12:41:00 AM,01/06/2020 05:10:00 PM,hubNashville Community,NaN,NaN,...,False,NaN,False,Columbia Ave,NASHVILLE,20.0,37209,36.152000,-86.872080,"(36.152, -86.87208)"
68533,286843,Closed,"Streets, Roads & Sidewalks",Street Lighting,Report Outage,12/31/2019 12:43:00 AM,01/09/2020 03:32:00 PM,Mobile App,NaN,NaN,...,False,NaN,False,6437 US-70,NASHVILLE,20.0,37209,36.141966,-86.876172,"(36.14196647, -86.87617231)"


## Looked at null values

In [7]:
mapped_loc_nulls= hub_data['Mapped Location'].isnull()

In [8]:
mapped_loc_nulls

0          True
1          True
2          True
3          True
4         False
          ...  
348666    False
348667    False
348668    False
348669    False
348670    False
Name: Mapped Location, Length: 348671, dtype: bool

In [9]:
mapped_loc_nulls.value_counts()

False    284188
True      64483
Name: Mapped Location, dtype: int64

## Convert to DateTime

In [ ]:
t1 = pd.to_datetime(hub_data['Date / Time Opened'])
t2 = pd.to_datetime(hub_data['Date / Time Closed'])
hub_data['time_taken'] = t1-t2
# move time_taken column to after time_closed
col_name = 'time_taken'
hub_time = hub_data.pop(col_name)
hub_data.insert(7, col_name, hub_time)


In [ ]:
hub_data['Date / Time Opened'] = pd.to_datetime(hub_data['Date / Time Opened'])

In [ ]:
hub_data['Date / Time Closed'] = pd.to_datetime(hub_data['Date / Time Closed'])

In [ ]:
hub_data.head(5)

## Create a year column

In [ ]:
##hub_data[['Year Requested']] = hub_data['Date / Time Opened'].str.split('-', n=1, expand=True)
##df['date'] = pd.to_datetime(df['date'])
##df['year'], df['month'] = df['date'].dt.year, df['date'].dt.month

hub_data['Year Requested'] = hub_data['Date / Time Opened'].dt.year

In [ ]:
hub_data.head()

## Look at counts in RequestType column

In [ ]:
hub_data['Request Type'].value_counts()

In [ ]:
hub_data['Subrequest Type'].value_counts()

## Look at counts in Status column

In [ ]:
hub_data['Status'].value_counts()

## Isolated COVID-19 requests

In [ ]:
covid_19_req = hub_data[hub_data['Request Type'] == 'COVID-19']
covid_19_req

## Subset dataframe with Subrequest for 'Dead Animal Pickup'

In [ ]:
animal_pickup = hub_data[(hub_data['Subrequest Type'].notnull())&(hub_data['Subrequest Type']=='Dead Animal Pickup')]

In [ ]:
animal_pickup.shape

In [ ]:
animal_pickup

## Subset dataframe with Subrequest for 'Debris'

In [ ]:
debris_sub = hub_data[(hub_data['Subrequest Type'].notnull())&(hub_data['Subrequest Type']=='Debris')]

In [ ]:
debris_sub.head()

## Map COVID-19 by zip code = requests, violations,  questions

### Create a geometry column

In [ ]:
hub_data['Geometry'] = hub_data.apply(lambda x: Point((float(x.Longitude), 
                                                         float(x.Latitude))), 
                                        axis=1)


In [ ]:
hub_data_geo = gpd.GeoDataFrame(hub_data,  
                           geometry = hub_data['Geometry'])
hub_data_geo

In [ ]:
print(hub_data_geo.crs) ## How do I set the crs?

## Bobcat 1
- Dead animal pickup and debris (for example) - make variable dataframes
- Count of dead animal pickup (or other categories), made histogram or other visual, count requests by zip 
- compare between different events (covid, tornado, control) and time of day. 
- Use this info to inform resource allocation.

## Bobcat 2
- Aggregate count of subrequest type by hour/month 
- How many of each (sub)request type are open at a given time to help staff forecast when heavy request load will occur.
- Consider dividing into 6hr chunks/bins for unit of analysis (morning, afternoon, evening, night)

## Bobcat 3
- Visualize geographical differences across time to completion/request type/other factors. 
- Identify spikes/outliers/trends across different dimensions (time/request type/volume/etc.). 
- Intention to help direct resources. Which area has the most activity to direct staff appropriately. 